In [47]:
import time
from pythonosc import udp_client, dispatcher, osc_server
from pythonosc.osc_message_builder import OscMessageBuilder
import threading
import librosa

librosa.hz_to_note(440.0)
c_natural = librosa.note_to_hz('C4')
c_sharp = librosa.note_to_hz('C#4')

sc_host = "127.0.0.1"
sc_port = 57110

print("=" * 70)
print("PYTHON-OSC TUTORIAL")
print("=" * 70)


PYTHON-OSC TUTORIAL


In [48]:
# =============================================================================
# SECTION 1: Understanding OSC Messages
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 1: What is an OSC Message?")
print("=" * 70)

"""
An OSC message has two parts:
1. ADDRESS PATTERN - like a URL path: /synth/frequency
2. ARGUMENTS - the data: 440, "hello", 0.5

Think of it like calling a function:
    /synth/play(frequency=440, amplitude=0.5)
"""

print("""
OSC Message Structure:
    Address: /synth/frequency
    Arguments: [440]

This means: "Set the frequency of 'synth' to 440"
""")


SECTION 1: What is an OSC Message?

OSC Message Structure:
    Address: /synth/frequency
    Arguments: [440]

This means: "Set the frequency of 'synth' to 440"



In [49]:
# =============================================================================
# SECTION 2: Your First OSC Client (Sending Messages)
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 2: Sending Your First OSC Message")
print("=" * 70)

host="127.0.0.1"
callback_port = 57120
# port = 59032
host= "localhost"

# Create a client that sends to localhost on port 57120 (SuperCollider default)
client = udp_client.SimpleUDPClient("127.0.0.1", callback_port)

print(f"Created OSC client targeting localhost:{callback_port}")
print("Sending message: /hello ['world']")

# Send a simple message
client.send_message("/hello", ["world"])

print("Message sent! (If SuperCollider is running, it received it)")
print("\nNote: Nothing happens yet because SC doesn't have a /hello handler")


SECTION 2: Sending Your First OSC Message
Created OSC client targeting localhost:57120
Sending message: /hello ['world']
Message sent! (If SuperCollider is running, it received it)

Note: Nothing happens yet because SC doesn't have a /hello handler


In [50]:

# =============================================================================
# SECTION 3: Controlling SuperCollider Synths
# =============================================================================
host = "127.0.0.1"
callback_port = 57110

# Create a client that sends to localhost on port 57120 (SuperCollider default)
client = udp_client.SimpleUDPClient(host, callback_port)

# Play a tone - NOTE: parameters come in key-value PAIRS
client.send_message("/s_new", ["default", -1, 0, 0, "freq", 440, "amp", 0.3])
print("Playing 440Hz tone...")
time.sleep(2)

# Stop all synths
client.send_message("/g_freeAll", [0])
print("Stopped")



Playing 440Hz tone...
Stopped


In [51]:
# =============================================================================
# SECTION 4: Playing Multiple Notes
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 4: Playing a Melody")
print("=" * 70)

# melody = [440, 494, 523, 587, 659, 698, 784, 880]  # A4 to A5
note_names = ["C4", "E4", "G4", "C5", "E5", "G5"]
melody = [librosa.note_to_hz(i) for i in note_names]

# input("\nPress ENTER to play a scale...")

for freq, name in zip(melody, note_names):
    print(f"Playing {name} ({freq}Hz)")
    client.send_message("/s_new", ["default", -1, 0, 0, "freq", freq, "amp", 0.5])
    time.sleep(0.1)
    client.send_message("/g_freeAll", [0])
    time.sleep(0.1)

print("Scale complete!")



SECTION 4: Playing a Melody
Playing C4 (261.6255653005986Hz)
Playing E4 (329.6275569128699Hz)
Playing G4 (391.99543598174927Hz)
Playing C5 (523.2511306011972Hz)
Playing E5 (659.2551138257398Hz)
Playing G5 (783.9908719634985Hz)
Scale complete!


In [52]:
# =============================================================================
# SECTION 5: Controlling Synth Parameters in Real-Time
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 5: Real-Time Parameter Control")
print("=" * 70)

print("""
We can control a running synth by:
1. Starting it with a specific node ID
2. Sending /n_set messages to change its parameters

Let's create a synth that sweeps from low to high frequency.
""")

# input("\nPress ENTER to hear a frequency sweep...")

# Start a synth with node ID 1000
client.send_message("/s_new", ["default", 1000, 0, 0, "freq", 200, "amp", 0.2])
print("Started synth with ID 1000")

# Sweep the frequency
for freq in range(500, 800, 1):
    client.send_message("/n_set", [1000, "freq", freq])
    print(f"Frequency: {freq}Hz")
    time.sleep(0.0005)

client.send_message("/n_set", [1000, "freq", 800])
time.sleep(1)

# Stop the specific synth
client.send_message("/n_free", [1000])
print("Synth stopped")



SECTION 5: Real-Time Parameter Control

We can control a running synth by:
1. Starting it with a specific node ID
2. Sending /n_set messages to change its parameters

Let's create a synth that sweeps from low to high frequency.

Started synth with ID 1000
Frequency: 500Hz
Frequency: 501Hz
Frequency: 502Hz
Frequency: 503Hz
Frequency: 504Hz
Frequency: 505Hz
Frequency: 506Hz
Frequency: 507Hz
Frequency: 508Hz
Frequency: 509Hz
Frequency: 510Hz
Frequency: 511Hz
Frequency: 512Hz
Frequency: 513Hz
Frequency: 514Hz
Frequency: 515Hz
Frequency: 516Hz
Frequency: 517Hz
Frequency: 518Hz
Frequency: 519Hz
Frequency: 520Hz
Frequency: 521Hz
Frequency: 522Hz
Frequency: 523Hz
Frequency: 524Hz
Frequency: 525Hz
Frequency: 526Hz
Frequency: 527Hz
Frequency: 528Hz
Frequency: 529Hz
Frequency: 530Hz
Frequency: 531Hz
Frequency: 532Hz
Frequency: 533Hz
Frequency: 534Hz
Frequency: 535Hz
Frequency: 536Hz
Frequency: 537Hz
Frequency: 538Hz
Frequency: 539Hz
Frequency: 540Hz
Frequency: 541Hz
Frequency: 542Hz
Frequency: 5

In [58]:
# =============================================================================
# SECTION 6: Receiving OSC Messages (Server)
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 6: Receiving OSC Messages")
print("=" * 70)

print("""
So far we've only SENT messages. Now let's RECEIVE them.
This is useful for:
- Building controllers
- Receiving feedback from audio software
- Building two-way communication systems
""")

import time
from pythonosc import udp_client, dispatcher, osc_server
import threading
import librosa


# Define a handler function
def print_handler(address, *args):
    print(f"Received: {address} with args: {args}")



# Create a dispatcher and server- routes incoming messages to functions
callback_port = 5009
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/*", print_handler)
server = osc_server.ThreadingOSCUDPServer(("127.0.0.1", callback_port), dispatcher)
print("OSC Server listening on 127.0.0.1:5005")


# Start server in background thread
server_thread = threading.Thread(target=server.serve_forever)
server_thread.daemon = True
server_thread.start()

print("\nServer is running. Let's send it some messages...")



# Create a client to send TO our own server
test_client = udp_client.SimpleUDPClient("127.0.0.1", callback_port)

# Send various messages
test_client.send_message("/test", ["hello"])
time.sleep(0.1)
test_client.send_message("/frequency", [440])
time.sleep(0.1)
test_client.send_message("/synth/play", ["note", 60, "velocity", 100])
time.sleep(0.5)

print("\nServer received the messages above!")




SECTION 6: Receiving OSC Messages

So far we've only SENT messages. Now let's RECEIVE them.
This is useful for:
- Building controllers
- Receiving feedback from audio software
- Building two-way communication systems

OSC Server listening on 127.0.0.1:5005

Server is running. Let's send it some messages...
Received: /test with args: ('hello',)
Received: /frequency with args: (440,)
Received: /synth/play with args: ('note', 60, 'velocity', 100)

Server received the messages above!


In [43]:
# =============================================================================
# SECTION 7: Building a Simple Sequencer
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 7: Building a Simple 4-Step Sequencer")
print("=" * 70)

client = udp_client.SimpleUDPClient(sc_host, sc_port)

sequence = [
    {"freq": 440, "dur": 0.5},
    {"freq": 550, "dur": 0.5},
    {"freq": 660, "dur": 0.5},
    {"freq": 440, "dur": 1},
]

print("Sequence pattern: A4 - C#5 - E5 - A4")
# input("\nPress ENTER to play the sequence twice...")

for loop in range(2):
    print(f"\nLoop {loop + 1}")
    for step, note in enumerate(sequence):
        print(f"  Step {step + 1}: {note['freq']}Hz")
        client.send_message("/s_new", ["default", -1, 0, 0,
                                       "freq", note['freq'],
                                       "amp", 0.2])
        time.sleep(note['dur'])
        client.send_message("/g_freeAll", [0])
        time.sleep(0.05)

print("\nSequencer demo complete!")


SECTION 7: Building a Simple 4-Step Sequencer
Sequence pattern: A4 - C#5 - E5 - A4

Loop 1
  Step 1: 440Hz
  Step 2: 550Hz
  Step 3: 660Hz
  Step 4: 440Hz

Loop 2
  Step 1: 440Hz
  Step 2: 550Hz
  Step 3: 660Hz
  Step 4: 440Hz

Sequencer demo complete!


In [31]:


# =============================================================================
# SECTION 8: Advanced - Bundles (Timed Messages)
# =============================================================================
print("\n" + "=" * 70)
print("SECTION 8: OSC Bundles - Perfectly Timed Messages")
print("=" * 70)


print("""
OSC Bundles let you send multiple messages with precise timing.
All messages in a bundle are executed at the exact same time.
This is crucial for tight musical timing.
""")

from pythonosc import osc_bundle_builder, osc_message_builder

# Create a bundle
bundle = osc_bundle_builder.OscBundleBuilder(
    osc_bundle_builder.IMMEDIATELY
)

client = udp_client.SimpleUDPClient(sc_host, sc_port)

def create_synth_message(synth_name, note: str, amp: float,  node_id=-1, add_action=0, target=0, **params):
    """Create a SuperCollider /s_new message wrapped in a bundle."""
    ifreq = float(librosa.note_to_hz(note))
    msg = osc_message_builder.OscMessageBuilder(address="/s_new")
    msg.add_arg(synth_name)
    msg.add_arg(node_id)
    msg.add_arg(add_action)
    msg.add_arg(target)
    msg.add_arg("freq")
    msg.add_arg(ifreq)
    msg.add_arg("amp")
    msg.add_arg(amp)

    for param_name, param_value in params.items():
        msg.add_arg(param_name)
        msg.add_arg(param_value)

    # Wrap the message in a bundle
    single_msg_bundle = osc_bundle_builder.OscBundleBuilder(
        osc_bundle_builder.IMMEDIATELY
    )
    single_msg_bundle.add_content(msg.build()) # type: ignore[arg-type]

    return single_msg_bundle.build()

# Now this works:
bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
bundle.add_content(create_synth_message("default", note="C4", amp=0.15))
bundle.add_content(create_synth_message("default", note="E4", amp=0.15))
bundle.add_content(create_synth_message("default", note="G4", amp=0.15))

# bundle.add_content(create_synth_message("default", note="f4", amp=0.15))

bundle.add_content(create_synth_message("default", note="A4", amp=0.15))
bundle.add_content(create_synth_message("default", note="B4", amp=0.15))

# Send the bundle
sub_bundle = bundle.build()
client._sock.sendto(sub_bundle.dgram, (sc_host, sc_port))

print("Playing A major chord (A-C#-E)")
time.sleep(2)
client.send_message("/g_freeAll", [0])


SECTION 8: OSC Bundles - Perfectly Timed Messages

OSC Bundles let you send multiple messages with precise timing.
All messages in a bundle are executed at the exact same time.
This is crucial for tight musical timing.

Playing A major chord (A-C#-E)
